In [5]:
import pandas as pd
import numpy as np
import itertools
import pprint
import re
import requests
import io
import time

import matplotlib.pyplot as plt
import seaborn as sns

from pymatgen import MPRester
from pymatgen.io.cif import CifBlock,CifFile
from pymatgen.core.structure import Structure
from pymatgen.core.lattice import Lattice

%matplotlib inline

/Users/mengzhao/miniconda2/envs/1_springboard_ds_track/lib/python3.6/site-packages/pymatgen/io/cif.py:37: UserWarning:

Please install optional dependency pybtex if youwant to extract references from CIF files.



In [6]:
def file_lengthy(f):
    """ count the number of lines in file handle
    Parameters
    ----------
    f : file handle
        file to read from
    Returns
    -------
    num_lines : int
        returns the number of lines in the file.
    --------
    """   
    for i, l in enumerate(f):
        pass
    return int(i + 1)

def read_ase_xyz(fin):
    """ read a xyz file created using ASE from file handle
    Parameters
    ----------
    fin : file handle
        file to read from
    Returns
    -------
    lattice_vector: 3*3 numpy matrix
    coords: cartesian coordinates of the system natoms*3 array
    species: a list of elements in the system
    --------
    """
    # count the number of atoms
    natoms = file_lengthy(fin) - 6
    
    # cursor returns to the first line of file   
    fin.seek(0)
    
    # skip the first three lines in ase_xyz file
    next(fin)
    next(fin)
    next(fin)
    
    lattice_vector = np.zeros([3, 3], dtype="float64")
    
    for vector in lattice_vector:
        line = fin.readline().split()
        vector[:] = list(map(float, line[1:4]))

    coords = np.zeros([natoms, 3], dtype="float64")
    species = []
    for x in coords:
        line = fin.readline().split()
        species.append(line[4])
        x[:] = list(map(float, line[1:4]))

    return lattice_vector, coords, species

def read_pymatgen_cif(stringIO, natoms):
    """ read structure information from string with pymatgen cif format
    Parameters
    ----------
    stringIO : io.StringIO with pymatgen cif format
    Returns
    -------
    lattice_vector: 3*3 numpy matrix
    coords: cartesian coordinates of the system natoms*3 array
    species: a list of elements in the system
    --------
    """
    # count the number of atoms
    natoms = file_lengthy(stringIO) - 26
    
    # cursor returns to the first line of file   
    stringIO.seek(0)
    
    # skip the first three lines in pymatgen_cif file
    next(stringIO)
    next(stringIO)
    next(stringIO)

    a = float(stringIO.readline().split()[1])
    b = float(stringIO.readline().split()[1])
    c = float(stringIO.readline().split()[1])
    
    alpha = float(stringIO.readline().split()[1])
    beta = float(stringIO.readline().split()[1])
    gamma = float(stringIO.readline().split()[1])
    
    lattice_vector = Lattice.from_parameters(a, b, c, alpha, beta, gamma) 
    lattice_vector_matrix = lattice_vector._matrix
    
    # skip uncessary rows
    for i in range(1,18):
        next(stringIO)

    coords = np.zeros([natoms, 3], dtype="float64")
    species = []
    for x in coords:
        line = stringIO.readline().split()
        species.append(line[0])
        x[:] = list(map(float, line[3:6]))

    return lattice_vector_matrix, coords, species

def read_poscar(fin):
    """ read a poscar file 
    Parameters
    ----------
    fin : file handle
        file to read from
    Returns
    -------
    lattice_vector: 3*3 numpy matrix
    coords: cartesian coordinates of the system natoms*3 array
    species: a list of elements in the system
    --------
    """
    # count the number of atoms
    natoms = file_lengthy(fin) - 8
    
    # cursor returns to the first line of file   
    fin.seek(0)
    
    # skip the first two lines in poscar
    next(fin)
    next(fin)
    
    lattice_vector = np.zeros([3, 3], dtype="float64")
    
    for vector in lattice_vector:
        line = fin.readline().split()
        vector[:] = list(map(float, line[0:3]))
        
    species = []   
    species_line = fin.readline().split()
    for s in species_line:
        species.append(s)
        
    species_nums = []
    species_nums_line = fin.readline().split()
    for num in species_nums_line:
        species_nums.append(int(num))
    
    species_corr = []
    for index, s in enumerate(species):
        for i in range(species_nums[index]):
            species_corr.append(s)
        
    # skip the next one line in poscar
    next(fin)      

    coords = np.zeros([natoms, 3], dtype="float64")
    for x in coords:
        line = fin.readline().split()
        x[:] = list(map(float, line[0:3]))

    return lattice_vector, coords, species_corr

# 1. gathering and wrangling datasets from NOMAD kaggle repository

In [7]:
start = time.time()

In [ ]:
# data_k has 2400 entries in total without any missing value
data_k= pd.read_csv('./data_part1/data.csv')
data_k.head()

In [ ]:
# initialize three lists
materials_ids = []
stru_list = []
formulas = []

# convert the geometry.xyz data into a pymatgen structure object
for materials_id in data_k["id"]:
    with open("./data_part1/data/"+ str(materials_id) + "/geometry.xyz") as f:
        lattice_vector, coords, species = read_ase_xyz(f)

    materials_ids.append(materials_id)
    stru = Structure(lattice_vector, species, coords, coords_are_cartesian=True)
    formula = stru.composition.reduced_formula
    stru_list.append(stru)
    formulas.append(formula)

stru_df = pd.DataFrame({"id":materials_ids,
                        "structure":stru_list,
                        "formula":formulas
                      })

stru_df.head()

In [ ]:
# join data_k and stru_df based on id

data_k_processed = data_k.merge(stru_df,on ='id')
data_k_processed.head()

In [ ]:
# drop uncessary columns as structure objects in structure columns contains
# information on number of total atoms, percent of atoms, lattices vector, angles and so on

data_k_processed = data_k_processed[["formula","structure","spacegroup",
                                     "formation_energy_ev_natom",
                                     "bandgap_energy_ev"]]
data_k_processed.head()

# 2. gathering and wrangling datasets from materials project database 

In [ ]:
api_key = "UFIyGHZh8JP5ikew"

# initializes the REST adaptor. Put your own API key in.
a = MPRester(api_key)
 
# get entries for desired chemical systems
entries_1 = a.get_entries_in_chemsys(['Al','Ga','In','O',
                                    'Mo','Zr','W', 'Ta',
                                    'Sb','Zn','Sn','Ti',
                                    'Ce'])

entries_2 = a.get_entries_in_chemsys(['O','Fe','Co','Cu',
                                    'Ni','Mn','Pt','Pd',
                                    'Ir','Ru'])
entries = entries_1 + entries_2

# print(entries)
mp_ids = []
for entry in entries:    
    # considering metal oxides at least one metal element contained 
    if entry.composition.to_data_dict["nelements"] > 1 and 'O' in entry.composition.as_dict().keys():       
        mp_ids.append(entry.entry_id)

In [ ]:
# initialize a list for storing detailed information of the entries
m = []
for mp_id in mp_ids:

    # get the relavent chemical properties based on the entires of interests
    mp_entry = requests.get("https://www.materialsproject.org/rest/v2/materials/"+
                             mp_id +"/vasp?API_KEY="+ api_key)
    
    mp_entry_json = mp_entry.json()
    m.append(mp_entry_json['response'][0])

In [ ]:
# creates a pandas dataframe using response json data
r = pd.DataFrame(m)
r_materials = r[['material_id','spacegroup','pretty_formula',
                 'unit_cell_formula','cif','band_gap',
                 'formation_energy_per_atom']]

# extract the spacegroup number of the materials
r_materials['spacegroup'] = r_materials['spacegroup'].apply(lambda x: x['number'])

# extract number_of_total_atoms of the materials
r_materials['number_of_total_atoms'] = r_materials['unit_cell_formula'].apply(lambda x: sum(x.values()))

# rename the columns to be consistent with the data_part1 from kaggle
r_materials.rename(columns={'formation_energy_per_atom':'formation_energy_ev_natom',
                            'band_gap':'bandgap_energy_ev',
                            'material_id':'id'}, 
                            inplace=True)

r_materials.head()

In [ ]:
cif_string = r_materials['cif'].tolist()
mp_stru_list = []
materials_mp_ids = []

for index, cif_s in enumerate(cif_string):
    
    materials_mp_id = r_materials['id'][index]
    materials_mp_ids.append(materials_mp_id)
    
    natoms = r_materials['number_of_total_atoms'][index]
    
    cif_reading = io.StringIO(cif_s)
    lattice_vector,coords, species = read_pymatgen_cif(cif_reading, natoms)
    mp_stru = Structure(lattice_vector, species, coords, coords_are_cartesian=False)
    mp_stru_list.append(mp_stru)

mp_stru_df = pd.DataFrame({"id":materials_mp_ids,
                           "structure":mp_stru_list,
                         })
mp_stru_df.head()

In [ ]:
data_mp_processed = r_materials.merge(mp_stru_df,on ='id')
data_mp_processed.head()

In [ ]:
data_mp_processed =data_mp_processed[["pretty_formula","structure","spacegroup",
                                     "formation_energy_ev_natom",
                                     "bandgap_energy_ev"]]
data_mp_processed = data_mp_processed.rename(columns = {'pretty_formula':'formula'})
data_mp_processed.head()

# 3. gathering and wrangling relavent datasets from ICSD 

In [ ]:
# load datasets from icsd
data_icsd= pd.read_csv('./data_part3/properties_icsd.txt',sep=' ')

In [ ]:
# remain the entries for semiconductors 
# where the bandgap is larger than 0 but smaller or equal to 3.0 eV
# by checking data_icsd_m_bandgaps.info() there is no missing values

data_icsd_m_bandgaps = data_icsd[(data_icsd['bandgap'] > 0.0) 
                                 &(data_icsd['bandgap'] <= 3.0)
                                ]

# only remains filenames with two target values
data_icsd_m_bandgaps = data_icsd_m_bandgaps[['filename','delta_e','bandgap']]

# initialize the empty lists
stru_icsd_list = []
formulas_icsd = []
filenames = []
spacegroups = []

# convert the VASP POSCAR into a pymatgen structure object
for f_name in data_icsd_m_bandgaps["filename"]:
    with open("./data_part3/icsd-all/"+ str(f_name)) as f:
        try:
            lattice_vector, coords, species_corr = read_poscar(f)
            stru = Structure(lattice_vector, species_corr, coords, coords_are_cartesian=False)
            formula = stru.composition.reduced_formula
            spacegroup_symbol, international_num= stru.get_space_group_info()
            stru_icsd_list.append(stru)
            formulas_icsd.append(formula)
            spacegroups.append(international_num)
            filenames.append(f_name)
        except:
            print(f_name)

stru_icsd_df = pd.DataFrame({"filename":filenames,
                             "structure":stru_icsd_list,
                             "formula":formulas_icsd,
                             "spacegroup":spacegroups
                           })

stru_icsd_df.info()
stru_icsd_df.head()

In [ ]:
data_icsd_processed = data_icsd_m_bandgaps.merge(stru_icsd_df,on ='filename')
data_icsd_processed.head()

In [ ]:
data_icsd_processed = data_icsd_processed[["formula","structure","spacegroup",
                                           "delta_e","bandgap"]]
data_icsd_processed = data_icsd_processed.rename(columns = {'delta_e':'formation_energy_ev_natom',
                                                        'bandgap':'bandgap_energy_ev'})
data_icsd_processed.head()

# 4. gathering and wrangling datasets from OQMD

In [ ]:
data_oqmd = pd.read_csv('./data_part3/properties_oqmd.txt',sep=' ')

# OQMD dataset contains 'None' strings
data_oqmd = data_oqmd.replace('None', np.nan)
data_oqmd.info()

In [ ]:
# bandgap column in OQMD has the missing values
# we will emlimate those entries as bandgap is one of the target values

# elimintate entries with missing bandgaps in oqmd
data_oqmd = data_oqmd.dropna()

# convert bandgap columns to the type of float
data_oqmd['bandgap'] = data_oqmd['bandgap'].astype(float)

# remain the semiconductors where the bandgap is larger than 0 but smaller or equal to 3.0 eV
data_oqmd_m_bandgaps = data_oqmd[(data_oqmd['bandgap'] > 0.0) 
                                 &(data_oqmd['bandgap'] <= 3.0)
                                ]
# only remains filenames with two target values
data_oqmd_m_bandgaps = data_oqmd_m_bandgaps[['filename','delta_e','bandgap']]

# initialize the empty lists
stru_oqmd_list = []
formulas_oqmd = []
filenames_oqmd = []
spacegroups_oqmd = []

# convert the VASP POSCAR into a pymatgen structure object
for f_name in data_oqmd_m_bandgaps["filename"]:
    with open("./data_part3/oqmd-all/"+ str(f_name)) as f:
        try:
            lattice_vector, coords, species_corr = read_poscar(f)
            stru = Structure(lattice_vector, species_corr, coords, coords_are_cartesian=False)
            formula = stru.composition.reduced_formula
            spacegroup_symbol, international_num= stru.get_space_group_info()
            stru_oqmd_list.append(stru)
            formulas_oqmd.append(formula)
            spacegroups_oqmd.append(international_num)
            filenames_oqmd.append(f_name)
        except:
            print(f_name)

stru_oqmd_df = pd.DataFrame({"filename":filenames_oqmd,
                             "structure":stru_oqmd_list,
                             "formula":formulas_oqmd,
                             "spacegroup":spacegroups_oqmd
                           })

stru_oqmd_df.info()
stru_oqmd_df.head()

In [ ]:
data_oqmd_processed = data_oqmd_m_bandgaps.merge(stru_oqmd_df,on ='filename')
data_oqmd_processed.head()

In [ ]:
data_oqmd_processed = data_oqmd_processed[["formula","structure","spacegroup",
                                           "delta_e","bandgap"]]
data_oqmd_processed = data_oqmd_processed.rename(columns = {'delta_e':'formation_energy_ev_natom',
                                                            'bandgap':'bandgap_energy_ev'})
data_oqmd_processed.head()

# 5. concatenate four datasets together

In [4]:
# concatenate two datasets data_k_processed and data_mp_processed together

frames = [data_k_processed,data_mp_processed,data_icsd_processed,data_oqmd_processed]
data_complete = pd.concat(frames)

data_complete.head()
data_complete.info()

NameError: name 'data_k_processed' is not defined

# 7. data explorations

In [ ]:
boxplot = data_complete.boxplot(column=['formation_energy_ev_natom',
                                        'bandgap_energy_ev'])
plt.ylabel("energy in eV")
plt.title("Figure 1. boxplot of formation_energy_ev_natom and bandgap_energy_ev\n")

In [ ]:
# found two outliers which have unrelistic formaiton energies 
# we decide to drop these two entries
data_complete[data_complete['formation_energy_ev_natom'] > 250]

In [ ]:
data_complete = data_complete[data_complete['formation_energy_ev_natom'] < 250]

# box plot after removing two outliers
boxplot = data_complete.boxplot(column=['formation_energy_ev_natom',
                                        'bandgap_energy_ev'])
plt.ylabel("energy in eV")
plt.title("Figure 2. boxplot of formation_energy_ev_natom and bandgap_energy_ev\n" +
          "after removing two outliers\n")

In [ ]:
data_complete.info()

# Featurization

In [ ]:
from matminer.featurizers.conversions import StrToComposition,CompositionToOxidComposition
from matminer.featurizers.composition import ElementProperty,OxidationStates
from matminer.featurizers.structure import DensityFeatures

# features
X = data_complete[['formula','structure','spacegroup']]
X_formula_feat = StrToComposition().featurize_dataframe(X, "formula")
X_formula_feat.head()

# 6. save datasets into CSV files

In [ ]:
# save the proceessed complete dataset into a csv file 
data_complete.to_csv("./data_complete.csv", sep=',') 

# save the proceessed dataset separately into a csv file 
data_k_processed.to_csv("./data_k_processed.csv", sep=',') 
data_mp_processed.to_csv("./data_mp_processed.csv", sep=',') 
data_icsd_processed.to_csv("./data_icsd_processed.csv", sep=',') 
data_oqmd_processed.to_csv("./data_oqmd_processed.csv", sep=',') 

In [ ]:
# timer
end = time.time()
print("The data processing time is " + str((end-start)/3600) + " hours.")